# Quantum Random Number Generation Tutorial

True random number generation is a notoriously difficult problem. Many "random" generators today are actually pseudo-random, using a starting seed to spawning seemingly-random numbers that are actually a repeatable function of that seed. Most true random number generations are based on measurements of some natural phenomenon, such as atmospheric noise or atomic decay. 
(You can read more about it [here]( https://en.wikipedia.org/wiki/Random_number_generation).) 

Quantum random number generators (QRNGs) are truly random. The quantum algorithm for random number generation is one of the simplest applications of quantum computing principles, requiring very few qubits to run.

In this tutorial you will:
* learn about quantum random number generation and the principles behind it,
* implement a variety of QRNGs with equal probability of any given number,
* implement a single-bit QRNG with weighted probabilities of generated bits.

Let's go!

## Introduction

Recall from the [Qubit](../Qubit/Qubit.ipynb) tutorial that a qubit state $|\psi\rangle$ is defined via the basis states $|0\rangle$ and $|1\rangle$ as:

$$|\psi\rangle = \begin{bmatrix} \alpha \\ \beta \end{bmatrix} = \alpha|0\rangle + \beta|1\rangle\text{, where }|\alpha|^2 + |\beta|^2 = 1$$

We call $\alpha$ and $\beta$ the **amplitudes** of states $|0\rangle$ and $|1\rangle$, respectively. 
When $|\psi\rangle$ is measured in the $\{|0\rangle, |1\rangle\}$ basis (the computational basis), the probabilities of the outcomes are defined based on the state amplitudes: there is a $|\alpha|^2$ probability that the measurement result will be $0$, and a $|\beta|^2$ probability that the measurement result will be $1$.

> For example, a qubit in state $\begin{bmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{bmatrix}$ will yield measurement results $0$ or $1$ with equal probability, while a qubit in state $\begin{bmatrix} \frac{1}{2} \\ \frac{\sqrt3}{2} \end{bmatrix}$ will yield measurement result $0$ only 25% of the time, and $1$ 75% of the time.

This is sufficient to implement a simple random number generator!

> Remember that you can refer to the [Single Qubit Gates tutorial](../SingleQubitGates/SingleQubitGates.ipynb) if you need a refresher on the various quantum gates and their usage in Q#.

## <span style="color:blue">Exercise 1</span>: Generate a single random bit

**Input:** None.

**Goal:** Generate a $0$ or $1$ with equal probability.

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Use the allocated qubit, apply a quantum gate to it, measure it and use the result to return a $0$ or $1$.
</details>

**Stretch goal:** Can you find a different way to implement this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    What are the different quantum states that produce $0$ and $1$ measurement results with the same probability? How would measuring the qubit in a different basis change the result? 
</details>


In [ ]:
%kata T1_RandomBit

operation RandomBit () : Int {
    use q = Qubit();
    // ...
    H(q);
    return M(q) == Zero ? 0 | 1;
}

Testing one random bit generation...
Test passed


Success!

## <span style="color:blue">Exercise 2</span>: Generate a random two-bit number

Now that you can generate a single random bit, you can use that logic to create random multi-bit numbers. Let's try first to make a two-bit number by combining two randomly generated bits.

**Input:** None.

**Goal:** Generate a random number in the range $[0, 3]$ with an equal probability of getting each of the four numbers.

**Stretch goal:** Can you do this without allocating qubits in this operation?

<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Remember that you can use the previously defined operations.
</details>

In [ ]:
%kata T2_RandomTwoBits

operation RandomTwoBits () : Int {
    // ...
    use qs = Qubit[2];
    H(qs[0]);
    H(qs[1]);
    let m0 = M(qs[0]) == Zero ? 0 | 1;
    let m1 = M(qs[1]) == Zero ? 0 | 1;
    return m0 * 2 + m1;
}

Testing two random bits generation...
Test passed


Success!

## <span style="color:blue">Exercise 3</span>: Generate a number of arbitrary size

Let's take it a step further and generate an $N$-bit number. 

> Remember that you can use previously defined operations in your solution.

**Input:** An integer $N$ ($1 \le N \le 10$).

**Goal:** Generate a random number in the range $[0, 2^N - 1]$ with an equal probability of getting each of the numbers in this range.

> Useful Q# documentation: 
> * [`for` loops](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/iterations), 
> * [mutable variables](https://docs.microsoft.com/azure/quantum/user-guide/language/typesystem/immutability), 
> * [exponents](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.powi).

In [ ]:
%kata T3_RandomNBits 

operation RandomNBits (N : Int) : Int {
    // ...
    mutable r = 0;
    for i in 0..N-1 {
        set r = r + (2^i) * RandomBit();
    }
    
    return r;
}

Testing N = 1...
Test passed for N = 1
Testing N = 2...
Test passed for N = 2
Testing N = 3...
Test passed for N = 3
Testing N = 10...
Test passed for N = 10


Success!

## <span style="color:blue">Exercise 4</span>: Generate a weighted bit!

In each of the above exercises, all generated numbers were equally likely. Now let's create an operation that will return a random bit with different probabilities of outcomes. 

> Remember that by setting amplitudes of basis states $\alpha$ and $\beta$, we can control the probability of getting measurement outcomes $0$ and $1$ when the qubit is measured.

**Input:** 
A floating-point number $x$, $0 \le x \le 1$. 

**Goal:** Generate $0$ or $1$ with probability of $0$ equal to $x$ and probability of $1$ equal to $1 - x$.

> Useful Q# documentation: 
> * [`Math` namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math)
> * [`ArcCos` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.arccos)
> * [`Sqrt` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.sqrt)


In [ ]:
%kata T4_WeightedRandomBit
open Microsoft.Quantum.Math;
operation WeightedRandomBit (x : Double) : Int {
    // ...
    let theta = ArcCos(Sqrt(x));
    use q = Qubit();
    Ry(2.0 * theta, q);
    return M(q) == Zero ? 0 | 1;
}

Testing generating zero with 0% probability...
Test passed for generating zero with 0% probability
Testing generating zero with 25% probability...
Test passed for generating zero with 25% probability
Testing generating zero with 50% probability...
Test passed for generating zero with 50% probability
Testing generating zero with 75% probability...
Test passed for generating zero with 75% probability
Testing generating zero with 100% probability...
Test passed for generating zero with 100% probability


Success!

## <span style="color:blue">Exercise 5</span>: Generate a random number between min and max

In exercise 3, we generated numbers in the range $[0, 2^N-1]$ $(1 \leq N \leq 10)$. Now let's create an operation that will return a random number in the range $[min, max]$. 

**Input:** 
Two integers $min$ and $max$ ($0 \leq min \leq max \leq 2^{10}-1$).

**Goal:** Generate a random number in the range $[min, max]$ with an equal probability of getting each of the numbers in this range.

> Useful Q# documentation: 
> * [`BitSizeI` function](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.bitsizei)

In [ ]:
%config dump.basisStateLabelingConvention = "Bitstring"
%config dump.truncationThreshold = 0.01
%config dump.truncateSmallAmplitudes = true
%config  

Configuration key,Value
dump.basisStateLabelingConvention,"""Bitstring"""
dump.truncationThreshold,0.01
dump.truncateSmallAmplitudes,true


In [ ]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;
operation PrepareRandomStates(qs: Qubit[], K: Int): Unit {
    //let bs = BitSizeI(K-1);
    //use qs = Qubit[bs];
    
    if (K == 1) {
    } elif (K == 2) { // 1 qubit
        H(qs[0]);
    } elif (K == 3) { // 2 qubit
        let ang = ArcTan(Sqrt(2.0)) + PI();
        Ry(ang * 2.0, qs[0]);
        Controlled H([qs[0]], qs[1]);
    } elif (K == 4) { // 2 qubit
        H(qs[0]);
        H(qs[1]);
    } elif (K == 5) { // 3 qubit
        let ang = ArcTan(0.5) + PI() * 0.5;
        Ry(ang * 2.0, qs[0]);
        Controlled H([qs[0]], qs[1]);
        Controlled H([qs[0]], qs[2]);
    } elif (K == 6) { // 3 qubit
        let phi = PI() * 0.5;
        let the = 2.0 * ArcTan(Sqrt(2.0));
        Ry(the, qs[0]);
        Ry(phi, qs[1]);
        Controlled H([qs[0]], qs[2]);
    } elif (K == 7) { // 3 qubit
        
    } elif (K == 8) { // 3 qubit
        
    } elif (K == 9) { // 4 qubit
        
    } elif (K == 10) { // 4 qubit
        
    }
    
    
    DumpMachine();
    //return qs;
}

PrepareRandomStates

In [ ]:
%simulate PrepareRandomStates qs=Qubit[3] K=6

Received invalid parameters. Please fix and try again:
 qs: Error converting value "Qubit[3]" to type 'System.Collections.Generic.List`1[Microsoft.Quantum.Simulation.Core.Qubit]'. Path '', line 1, position 10.


In [ ]:
operation PrepareRandomStatesDump() {
    
}

In [ ]:
%kata T5_RandomNumberInRange
open Microsoft.Quantum.Convert;

operation RandomNumberInRange (min : Int, max : Int) : Int {
    // ...
    //let N = max - min;
    //let r = RandomNBits(max) % N;
    //let maxB = BitSizeI(max);
    //let minB = BitSizeI(min);
    //let N = maxB - minB;
    //let r = RandomNBits(N);
    // Message(IntAsString(min) + ", " + IntAsString(max) + ", " 
    //     + IntAsString(BitSizeI(min)) + ", " + IntAsString(BitSizeI(max)));
    //let N = BitSizeI(max - min);
    //let r = RandomNBits(N) % max;
    // Message(IntAsString(min) + ", " + IntAsString(r));
    //let maxB = BitSizeI(max);
    //let minB = BitSizeI(min);
    //use qs = Qubit[max-1];
    //for i in 0..(max-2) {
    //    H(qs[i]);
    //}
    //
    //mutable r = 0;
    //for i in 0..max-2 {
    //    let mi = M(qs[i]) == Zero ? 0 | 1;
    //    set r = r + mi;
    //}
    //
    let K = max - min;
    let bs = BitSizeI(K);
    use qs = Qubit[bs];
    mutable r = 0;
    if (K == 1) {
    } elif (K == 2) { // 1 qubit
        H(qs[0]);
    } elif (K == 3) { // 2 qubit
        let ang = ArcTan(Sqrt(2.0)) + PI();
        Ry(ang * 2.0, qs[0]);
        Controlled H([qs[0]], qs[1]);
    } elif (K == 4) { // 2 qubit
        H(qs[0]);
        H(qs[1]);
    } elif (K == 5) { // 3 qubit
        let ang = ArcTan(0.5) + PI() * 0.5;
        Ry(ang * 2.0, qs[0]);
        Controlled H([qs[0]], qs[1]);
        Controlled H([qs[0]], qs[2]);
    } elif (K == 6) { // 3 qubit
        let phi = PI() * 0.5;
        let the = 2.0 * ArcTan(Sqrt(2.0));
        Ry(the, qs[0]);
        Ry(phi, qs[1]);
        Controlled H([qs[0]], qs[2]);
    } elif (K == 7) { // 3 qubit
        
    } elif (K == 8) { // 3 qubit
        
    } elif (K == 9) { // 4 qubit
        
    } elif (K == 10) { // 4 qubit
        
    }
    
    

    
    
    ResetAll(qs);
    
    return r + min;
}

Testing for min = 1 and max = 3...
Unexpected average of generated numbers. Expected between 1.9225403330758517 and 2.0774596669241485, got 1
Unexpected average of generated numbers. Expected between 1.9225403330758517 and 2.0774596669241485, got 1
Unexpected average of generated numbers. Expected between 1.9225403330758517 and 2.0774596669241485, got 1


Failed to generate sufficiently random integer
Try again!


## What's Next?
We hope you enjoyed this tutorial on quantum random number generation! If you're looking to learn more about quantum computing and Q#, here are some suggestions:
* The [Quantum Katas](https://github.com/microsoft/QuantumKatas/) are sets of programming exercises on quantum computing that can be solved using Q#. They cover a variety of topics, from the basics like the concepts of superposition and measurements to more interesting algorithms like Grover's search.
* For another look at quantum random number generation, you can check out the [Microsoft Learn module "Create your first Q# program by using the Quantum Development Kit"](https://docs.microsoft.com/learn/modules/qsharp-create-first-quantum-development-kit/1-introduction).